## Walmart Baseline

보통 점수가 잘 안나는 이유는??
1. 모델을 잘못 선택. feature가 category의 성질을 띄고 있으면, tree 계열이 잘된다
2. column을 무작위로 추가. overfitting이 유발된다.

이를 해결하기 위해서는??
1. Data preprocessing에서 column 추가 -> data를 추가해준다
2. Data에 맞는 model을 선택하거나 option을 추가해준다.
하지만 Data를 추가해주는 방식이 당연히 더 잘된다!!


Regression Model로 MSE나 MAE로 평가한다.

이 대회의 경우, 공휴일일 때 weight값을 준다. 어떤 달의 공휴일에 장사가 더 잘되는지 (대회 예시에서는 크리스마스를 들고 있음)



In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
train = pd.read_csv('/content/drive/MyDrive/kaggle/Walmart_prediction/train.csv.zip')
train

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [3]:
test = pd.read_csv('/content/drive/MyDrive/kaggle/Walmart_prediction/test.csv.zip')
test

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False
...,...,...,...,...
115059,45,98,2013-06-28,False
115060,45,98,2013-07-05,False
115061,45,98,2013-07-12,False
115062,45,98,2013-07-19,False


In [4]:
train['Date'] = pd.to_datetime(train['Date'])
train['Date']

0        2010-02-05
1        2010-02-12
2        2010-02-19
3        2010-02-26
4        2010-03-05
            ...    
421565   2012-09-28
421566   2012-10-05
421567   2012-10-12
421568   2012-10-19
421569   2012-10-26
Name: Date, Length: 421570, dtype: datetime64[ns]

ISO 달력은 널리 사용되는 그레고리력의 변형입니다.

ISO 연도는 52나 53개의 완전한 주로 구성되고, 주는 월요일에 시작하여 일요일에 끝납니다. ISO 연도의 첫 번째 주는 그 해의 (그레고리) 달력에서 목요일이 들어있는 첫 번째 주입니다. 이것을 주 번호 1이라고 하며, 그 목요일의 ISO 연도는 그레고리 연도와 같습니다.

예를 들어, 2004년은 목요일에 시작되므로, ISO 연도 2004의 첫 주는 월요일, 2003년 12월 29일에 시작하고, 일요일, 2004년 1월 4일에 끝납니다.

Week를 고려하는 이유는 공휴일이 1달에 많을 때 어떤 week에(첫번째 week 또는 마지막 week에) 공휴일이 많은지 확인해줄 수 있음 !!

In [5]:
train['Year'] = train['Date'].dt.year
train['Month'] = train['Date'].dt.month
train['Day'] = train['Date'].dt.day
train['Week'] = train['Date'].dt.week

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.


In [6]:
train.head(5)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Year,Month,Day,Week
0,1,1,2010-02-05,24924.50,False,2010,2,5,5
1,1,1,2010-02-12,46039.49,True,2010,2,12,6
2,1,1,2010-02-19,41595.55,False,2010,2,19,7
3,1,1,2010-02-26,19403.54,False,2010,2,26,8
4,1,1,2010-03-05,21827.90,False,2010,3,5,9


In [9]:
train.groupby('Day')['Weekly_Sales'].mean() # 날마다 평균 판매량
# 사람들 행동 패턴을 확인 + 회사 프로모션 영향을 볼 수 있다. 11번가 11절 같은거!!

Day
1     15438.693561
2     16282.825224
3     16243.591840
4     16262.123390
5     16152.185352
6     16540.152518
7     15851.075764
8     16026.260550
9     16563.236628
10    16684.753968
11    15946.176870
12    15769.983633
13    15358.628217
14    14843.544775
15    15476.565690
16    16400.916009
17    16890.881861
18    15914.848035
19    15634.121093
20    15236.067426
21    14987.074671
22    15696.588300
23    17183.530448
24    17732.369226
25    16816.726353
26    16742.479327
27    14908.182971
28    15056.847856
29    15117.011770
30    15038.411334
31    14833.557736
Name: Weekly_Sales, dtype: float64

### Month는 왜 성능이 확 좋아지는가??
카테고리가 12개로 tree 모델에서 잘 된다. 그리고 month feature의 영향력이 이 데이터에서 크다 !!

In [8]:
train.groupby('Month')['Weekly_Sales'].mean() # 달마다 평균 판매량

Month
1     14126.075111
2     16008.779217
3     15416.657597
4     15650.338357
5     15776.337202
6     16326.137002
7     15861.419650
8     16062.516933
9     15095.886154
10    15243.855576
11    17491.031424
12    19355.702141
Name: Weekly_Sales, dtype: float64

In [10]:
train['Month'].value_counts() # 갯수 확인

4     41332
7     40980
3     38451
10    38362
9     38339
8     38169
6     38137
2     35526
5     35314
12    29802
11    23613
1     23545
Name: Month, dtype: int64

In [12]:
train_2 = train.drop(columns=['Date','Weekly_Sales'])
train_2

,Store,Dept,IsHoliday,Year,Month,Day,Week
0,1,1,False,2010,2,5,5
1,1,1,True,2010,2,12,6
2,1,1,False,2010,2,19,7
3,1,1,False,2010,2,26,8
4,1,1,False,2010,3,5,9
...,...,...,...,...,...,...,...
421565,45,98,False,2012,9,28,39
421566,45,98,False,2012,10,5,40
421567,45,98,False,2012,10,12,41
421568,45,98,False,2012,10,19,42


In [14]:
test['Date'] = pd.to_datetime(test['Date'])
test['Year'] = test['Date'].dt.year
test['Month'] = test['Date'].dt.month
test['Day'] = test['Date'].dt.day
test['Week'] = test['Date'].dt.week
test.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


,Store,Dept,Date,IsHoliday,Year,Month,Day,Week
0,1,1,2012-11-02,False,2012,11,2,44
1,1,1,2012-11-09,False,2012,11,9,45
2,1,1,2012-11-16,False,2012,11,16,46
3,1,1,2012-11-23,True,2012,11,23,47
4,1,1,2012-11-30,False,2012,11,30,48


In [15]:
test_2 = test.drop(columns=['Date'])
test_2

,Store,Dept,IsHoliday,Year,Month,Day,Week
0,1,1,False,2012,11,2,44
1,1,1,False,2012,11,9,45
2,1,1,False,2012,11,16,46
3,1,1,True,2012,11,23,47
4,1,1,False,2012,11,30,48
...,...,...,...,...,...,...,...
115059,45,98,False,2013,6,28,26
115060,45,98,False,2013,7,5,27
115061,45,98,False,2013,7,12,28
115062,45,98,False,2013,7,19,29


n_jobs 옵션 : cpu 사용 개수를 의미

In [16]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_jobs = -1)
rf.fit(train_2, train['Weekly_Sales'])
result = rf.predict(test_2)

pd.Series에 대한 설명 !!

In [17]:
pd.Series(rf.feature_importances_, index=train_2.columns).sort_values(ascending=True) # 변수 중요도 확인

IsHoliday    0.001214
Year         0.005643
Month        0.008254
Day          0.012280
Week         0.043356
Store        0.370662
Dept         0.558591
dtype: float64

제출하기 !!

In [18]:
sub = pd.read_csv('/content/drive/MyDrive/kaggle/Walmart_prediction/sampleSubmission.csv.zip')
sub

,Id,Weekly_Sales
0,1_1_2012-11-02,0
1,1_1_2012-11-09,0
2,1_1_2012-11-16,0
3,1_1_2012-11-23,0
4,1_1_2012-11-30,0
...,...,...
115059,45_98_2013-06-28,0
115060,45_98_2013-07-05,0
115061,45_98_2013-07-12,0
115062,45_98_2013-07-19,0


In [19]:
sub['Weekly_Sales'] = result
sub

,Id,Weekly_Sales
0,1_1_2012-11-02,35890.4728
1,1_1_2012-11-09,21206.1537
2,1_1_2012-11-16,19391.4403
3,1_1_2012-11-23,20559.5592
4,1_1_2012-11-30,27896.0392
...,...,...
115059,45_98_2013-06-28,671.6377
115060,45_98_2013-07-05,676.5325
115061,45_98_2013-07-12,730.2526
115062,45_98_2013-07-19,769.1555


index가 true일 경우 index가 포함되어서 파일이 생성되기 때문에 제외시켜서 csv 파일로 생성해야됨!

In [20]:
sub.to_csv('/content/drive/MyDrive/kaggle/Walmart_prediction/Submission.csv', index=False)

Public score는 공개된 점수로 test data 중 30%로 평가 !!
대회가 끝난 뒤에 Private score가 공개됨. 이 때는 test data 중 70%로 평가됨 !!


Private score가 잘 나오기 위해서는 overfitting 조심!! 가볍고 단순해야 된다. 보통 Cross Validation으로 평가한다 !!

### 숙제
1. baseline(날짜 column 추가 없이) 결과 확인(1.37 정도)
2. 날짜 추가해서 결과 확인 (0.5 이하) 최대한 점수 올려보기 !!
3. 어떤 날짜 column이 도움이 안되는지 찾아오기 !!
4. 질문 최대한 많이 준비해오기 !!